In [ ]:
# Seaborn and polars
import seaborn as sns
import polars as pl
import plotly.express as px

import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

In [ ]:
hwe = pl.read_csv("out.hwe", separator="\t")
hwe.head()

In [ ]:
# Histogram
hwe = pl.read_csv("out.hwe", separator="\t")
obs = hwe["OBS(HOM1/HET/HOM2)"].str.split("/").list.eval(pl.element().str.to_integer())
obs = obs.list.to_struct()
hwe = hwe.with_columns(obs).unnest("OBS(HOM1/HET/HOM2)")
# Rename field_0, field_1, field_2 to HOM1, HET, HOM2
hwe = hwe.rename(
    { "field_0": "HOM1", "field_1": "HET", "field_2": "HOM2" }
)

In [ ]:
# New colum that is HOM1 + HOM2
hwe = hwe.with_columns(
    (pl.col("HOM1") + pl.col("HOM2")).alias("HOM")
)
hwe

In [ ]:
# Ratio of HET to HOM
hwe = hwe.with_columns(
    (pl.col("HET") / pl.col("HOM")).alias("HET/HOM")
)

In [ ]:
# Histogram of the ratio in plotly with 1000 bins
fig = px.histogram(hwe.to_pandas(), x="HET/HOM", nbins=1000)
fig.show()


In [ ]:
# Per-individual het
per_sample_het = pl.read_csv("out.het", separator="\t")
# Get actual het count normalized to total
per_sample_het = per_sample_het.with_columns(
    (pl.col("N_SITES") - pl.col("O(HOM)")).alias("HET")
)

# Normalize to N_SITES
per_sample_het = per_sample_het.with_columns(
    (pl.col("HET") / pl.col("N_SITES")).alias("HET_norm")
)

per_sample_het.head()

In [ ]:
fig = px.bar(
    per_sample_het.to_pandas(),
    x="INDV",
    y="F"
)
fig.update_layout(xaxis_type='category')
fig.show()

In [ ]:
fig = px.scatter(
    per_sample_het.to_pandas(),
    x="INDV",
    y="F"
)
fig.update_layout(xaxis_type='category')
fig.show()

In [ ]:
fig = px.scatter(
    per_sample_het.to_pandas(),
    x="N_SITES",
    y="O(HOM)",
    hover_name="INDV"


)
# fig.update_layout(xaxis_type='category')
fig.show()

In [ ]:
fig = px.scatter(
    per_sample_het.to_pandas(),
    x="INDV",
    y="HET_norm"
)
fig.update_layout(xaxis_type='category')
fig.show()